In [ ]:
"""Import modules and define fixed parameters."""

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
from dome_connector import ShipConnector
from cmcl_jobsender import JobSender
from copy import deepcopy
import re

local = True
if local:
    # local deployment
    DOME_URL = 'https://cmcl.dome40.io/'  # DOME platform URL
    API_KEY = "6f7d3ae990.fb00950be9a1439d80c6baf515e1f112"  # Identify user
    CMCL_URL = 'http://192.168.1.171:4242/'  # URL of CMCL server
    # UUID of DOME connector for ship data
    SHIP_CONN_UUID = "fb8490c8-a71a-42c4-bd67-6052cf347f2e"
else:
    # connect to platform
    DOME_URL = 'https://nextgen.dome40.io/'
    API_KEY = "3b35fa9591.bd62b7cddfe04b26916dd6eb95214094"
    CMCL_URL = 'https://theworldavatar.io/demos/ship-emission/'
    SHIP_CONN_UUID = "0a367029-0bf1-4c75-b5e2-182cd88bdf53"

In [ ]:
"""Handle pasring query parameter."""
import os
from urllib.parse import parse_qs

query_string = os.environ.get('QUERY_STRING', '')

params = parse_qs(query_string)
    
if 'mmsi' in params:    
    QUERY_MMSI = params['mmsi'][0]
else:
    raise Exception("MMSI parameter must be supplied in the form of ?mmsi=123456789 in the HTTP request.")

In [ ]:
# search DOME for ship location data - backend

# create connectors

ship_connector = ShipConnector(DOME_URL, API_KEY, SHIP_CONN_UUID)

DICT_SHIP_ALL = ship_connector.get_ship("AIS")

for mmsi in DICT_SHIP_ALL:
    DICT_SHIP_ALL[mmsi]['dates'] = pd.to_datetime(DICT_SHIP_ALL[mmsi]['date'])

ALL_MMSI = list(DICT_SHIP_ALL.keys())

SCOPE_ALL = {k: ship_connector.get_scope(v, min_d=0.) for k,v in DICT_SHIP_ALL.items()} # This does not enforce minimum size

# only include 1 ship by default

LIST_MMSI = [QUERY_MMSI]

def update_by_ship_selection(list_mmsi):
    
    global SCOPE_ALL, DICT_SHIP_ALL

    scope = ship_connector.combine_scope(SCOPE_ALL, list_mmsi) # Final scope should enforce minimum size

    dict_df_ship = {}
    list_timestep = []
    for mmsi in list_mmsi:
        dict_df_ship[mmsi]=DICT_SHIP_ALL[mmsi]
        list_timestep.extend([x.to_pydatetime() for x in DICT_SHIP_ALL[mmsi]['dates']])

    list_timestep = list(set(list_timestep)) # remove duplicate
    list_timestep.sort() # sort ascendingly
    
    return scope, dict_df_ship, list_timestep

SCOPE, DICT_DF_SHIP, LIST_TIMESTEP = update_by_ship_selection(LIST_MMSI)

# Initialise plot and ship selection

output_dome_ship = widgets.HTML(value=ship_connector.plot_ship(DICT_SHIP_ALL,SCOPE,LIST_MMSI),
                                layout=widgets.Layout(width = "100%", padding='10px', margin='0px 0px 20px 0px'))

checklist_ship = widgets.SelectMultiple(
    options=ALL_MMSI,
    value = LIST_MMSI,
    description='Ships',
    disabled=False,
    layout=widgets.Layout(width = "80%"),
    
)

In [ ]:
# submit job to CMCL server

from uuid import uuid4

job_sender = JobSender(CMCL_URL)

# Create a text area widget
text_area_job_label = widgets.Text(
    placeholder=f"""Sim{uuid4()}""",
    description="""Name of simulation: """,
    layout=widgets.Layout(flex_flow='row',width='70%',padding='10px 0px 0px 0px'),
    style={'description_width': 'initial'},
    disabled=False
)

slider_job_step = widgets.IntSlider(
    value=0,
    min=1,
    max=len(LIST_TIMESTEP),
    description="""Number of timesteps: """,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(flex_flow='row',width='70%'),
    style={'description_width': 'initial'},
    disabled=False
)

text_area_scope_min = widgets.Text(
    placeholder=f"({SCOPE['LAT']-SCOPE['dLAT']:.5f}, {SCOPE['LON']-SCOPE['dLON']:.5f})",
    description="""Minimum (Lat,Lon): """,
    layout=widgets.Layout(display='flex', flex_flow='column', align_items='center', width="70%"),
    style={'description_width': 'initial'},
    disabled=False
)

text_area_scope_max = widgets.Text(
    placeholder=f"({SCOPE['LAT']+SCOPE['dLAT']:.5f}, {SCOPE['LON']+SCOPE['dLON']:.5f})",
    description="""Maximum (Lat,Lon): """,
    layout=widgets.Layout(display='flex', flex_flow='column', align_items='center', width="70%"),
    style={'description_width': 'initial'},
    disabled=False
)

text_area_job_password = widgets.Password(
    placeholder="""******""",
    description="""Password: """,
    layout=widgets.Layout(flex_flow='row',width='70%'),
    style={'description_width': 'initial'},
    disabled=False
)

# Create a button widget for overwriting the scope
button_update = widgets.Button(
    description='Update',
    button_style='warning',
    tooltip='Update scope',
    icon='circle'
)

def click_button_update(b):
    
    global SCOPE, DICT_DF_SHIP, LIST_TIMESTEP, LIST_MMSI
    
    b.button_style = 'info'  # Change the button style
    
    with output_job:

        output_job.clear_output()
        # handle ship selection
        LIST_MMSI = list(checklist_ship.value)
        SCOPE, DICT_DF_SHIP, LIST_TIMESTEP = update_by_ship_selection(LIST_MMSI)
        
        # handle scope specification
        try:
            # parse user input
            match_min = re.findall(r'-?\d+\.?\d*', text_area_scope_min.value)
            min_lat, min_lon = float(match_min[0]), float(match_min[1])
            match_max = re.findall(r'-?\d+\.?\d*', text_area_scope_max.value)
            max_lat, max_lon = float(match_max[0]), float(match_max[1])
            # update scope
            fake_ship = pd.DataFrame({'lat':[min_lat, max_lat],
                                      'lon':[min_lon, max_lon]})
            SCOPE = ship_connector.get_scope(fake_ship, min_d=0.) # allow user to specify scope exactly
        except:
            pass
        
        # update front-end
        output_dome_ship.value=ship_connector.plot_ship(DICT_SHIP_ALL,SCOPE,LIST_MMSI)
        slider_job_step.max=len(LIST_TIMESTEP)
    
    b.button_style = 'warning'  # Change the button style

button_update.on_click(click_button_update)
        
# Create a button widget for reset the scope
button_reset = widgets.Button(
    description='Reset',
    button_style='warning',
    tooltip='Reset scope',
    icon='circle'
)

def click_button_reset(b):
    
    b.button_style = 'info'  # Change the button style
    
    with output_job:
        output_job.clear_output()
        global SCOPE, DICT_DF_SHIP, LIST_TIMESTEP, LIST_MMSI
        LIST_MMSI = [QUERY_MMSI]
        SCOPE, DICT_DF_SHIP, LIST_TIMESTEP = update_by_ship_selection(LIST_MMSI)
        output_dome_ship.value=ship_connector.plot_ship(DICT_SHIP_ALL,SCOPE,LIST_MMSI)
        checklist_ship.value=LIST_MMSI
    
    b.button_style = 'warning'  # Change the button style


button_reset.on_click(click_button_reset)

# Create a button widget for job submission
button_job = widgets.Button(
    description='Submit',
    button_style='warning',
    tooltip='Submit job',
    icon='circle'
)

def click_button_job(b):
    
    ob_style = b.button_style
    ob_description = b.description
    ob_icon = b.icon
    
    b.button_style = 'info'  # Change the button style
    b.description = 'Submitted!'  # Change the button description
    b.icon = 'check'

    with output_job:
        output_job.clear_output()
        
        if len(text_area_job_label.value)>0:
            label = text_area_job_label.value
        else:
            label = text_area_job_label.placeholder
            
        try:
            num_step = slider_job_step.value
        except:
            print('Cannot parse number of steps, assume to be 0.')
            num_step = 0
        
        if num_step == 0:
            print('Number of timestep = 0 i.e. all timesteps will be simulated.')
            num_step = len(LIST_TIMESTEP)
        
        if text_area_job_password.value == os.getenv('NB_PW', uuid4()):
            for mmsi, df_ship in DICT_DF_SHIP.items():
                print(f'Sending ship data of MMSI:{mmsi}...')
                response = job_sender.add_ship_data(mmsi,df_ship)
            if response.status_code == 200:
                print('Ship data successfully sent.')
                print(job_sender.run_simulation_without_ship(label,SCOPE,LIST_TIMESTEP,num_step))
        else:
            print('No correct password is supplied.')
            print('*** MOCK SIMULATION OUTPUT ***')
            for mmsi, df_ship in DICT_DF_SHIP.items():
                print(f'Sending ship data of MMSI:{mmsi}...')
            print('Ship data successfully sent.')
            print(f'Running simulation {label}...')
            count = 0
            for t in LIST_TIMESTEP:
                if count < num_step:
                    print(f'Simulating {label} at {t}')
                    count = count + 1
            print('Complete.')

    b.button_style = ob_style  # Change the button style
    b.description = ob_description  # Change the button description
    b.icon = ob_icon

# Set the event handler for the button click event
button_job.on_click(click_button_job)

output_job = widgets.Output()
output_job.layout.overflow = 'auto'
output_job.layout.max_height = '100px'

In [ ]:
# final-layout

colour_bg="linear-gradient(to top left, rgba(131,195,141,0.3), rgba(200,200,200,0.3))"
colour_edge="rgba(124,203,219,1)"
css_edge="1px solid"
css = f"""
<style>
* {{
font-family: 'Verdana';
}}
.widget-label, .widget-button, .widget-text, .widget-int-text, .widget-float-text, .widget-dropdown, .widget-select, .widget-checkbox, .widget-radio, .widget-slider, .widget-progress {{
font-size: 12px;
color: #101010;
}}
.label_style{{
    border : {css_edge};
    width:auto;
    border-radius: 10px;
    font-size:20px;
    font-family: 'Verdana';
    font-weight:bold;
    color:black;
    text-align:center;
    border-color:{colour_edge};
    background: {colour_bg};
    padding: 10px 10px 10px 10px;
    margin: 0px 0px 20px 0px;
}}
.box_style{{
    border : {css_edge};
    border-radius: 20px;
    height: auto;
    max-height: 350px;
    border-color:{colour_edge};
    background: {colour_bg};
}}
.debug_style{{
    border : {css_edge};
    border-radius: 20px;
    height: auto;
    max-height: 350px;
    border-color: black;
    background: white;
}}
ol {{
font-family: 'Verdana';
font-size: 10px;
}}
ol li {{
margin-bottom: 0px;
padding: 0;
}}
</style>
"""
display(HTML(css))

label_dome_ship = widgets.HTML(f"<div>Location data of ship MMSI:{QUERY_MMSI}</div>")
label_dome_ship.add_class('label_style')

output_dome_ship.add_class('box_style')

vbox_dome_ship = widgets.VBox(
    [label_dome_ship, output_dome_ship],
    layout=widgets.Layout(width="50%",align_items='center', margin='20px'))

########################

label_job = widgets.HTML("<div>Configure and send simulation request</div>")
label_job.add_class('label_style')

vbox_scope = widgets.VBox(
    [text_area_scope_min, text_area_scope_max],
    layout=widgets.Layout(width="80%",align_items='center')
)

hbox_config_entry = widgets.HBox(
    [vbox_scope, checklist_ship],
    layout=widgets.Layout(width="100%",align_items='center', align_content='flex-start')
)

hbox_config_button = widgets.HBox(
    [button_update, button_reset],
    layout=widgets.Layout()
)

vbox_form = widgets.VBox(
    [hbox_config_entry, hbox_config_button,
     text_area_job_label, slider_job_step,
     text_area_job_password, button_job, output_job],
    layout=widgets.Layout(width="100%",align_items='center')
)

vbox_job = widgets.VBox(
    [label_job, vbox_form],
    layout=widgets.Layout(width="50%",align_items='center', margin='20px'))
vbox_form.add_class('box_style')

table = widgets.HBox([vbox_dome_ship, vbox_job],
                     layout=widgets.Layout(align_items='flex-start'), width='100%')

label_job = widgets.HTML('<div">DOME Showcase 1: Chemistry Knowledge Graph - Marine, Air Quality And Nanoparticles</div>',
                         layout=widgets.Layout(width='100%'))
label_job.add_class('label_style')

html_footer = widgets.HTML("""
<div style="text-align: right;">
<div style="display: inline-block; vertical-align: top;">
<a href="https://dome40.eu/">
<img src="https://dome40.eu/sites/default/files/DOME_LOGO_C.png" style="height: 30px;">
</a>
</div>
<div style="display: inline-block; vertical-align: top;">
<a href="https://cmcl.io/">
<img src="https://cmcl.io/wp-content/uploads/2024/07/astra_logo_large.png" style="height: 30px;">
</a>
</div>
</div>
""",
layout=widgets.Layout(width='20%'))

html_help = widgets.HTML("""
<div align="left" style="padding-top: 0px;padding-left: 20px;padding-bottom: 0px;"><b>How to use</b></div>
<ol>
<li>Specify the scope of simulation and select ships to be included in the simulation (use Ctrl and Shift). After that, click Update. The scope is automatically calculated from the ship selection if not explicitly specified. The visualisation should reflect the latest settings. Press Reset to revert to default settings.</li>
<li>Specify the name of the simulation and the number of timesteps to be simulated. Supply the correct password to perform simulation, otherwise only mock output messages will be shown.</li>
<li>Log messages should appear during the simulation. Once complete, the simulation result should be visible in the visualisation tool.</li>
</ol>
""",
layout=widgets.Layout(width='80%'))
html_help.add_class('box_style')

vbox_footer = widgets.HBox([html_help, html_footer], layout=widgets.Layout(align_items='flex-end'))

page = widgets.VBox([label_job, table, vbox_footer], label=widgets.Layout(align_items='center'))

display(page)